<a href="https://colab.research.google.com/github/WanjaSchaible/colab_py_shallow/blob/master/notebooks/RAGChatbot_groq_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Chatbot using Retrieval Augmented Generation
# Uses Groq Inference Platform to run LLM online

# TODO: benötigt groq API Key in GROQ_API_KEY und HuggingFace Token in HF_TOKEN (für Embedding Modell)
# TODO: erstelle Ordner pdfs und lade ein oder mehrere PDFs in diesen Ordner hoch

# IMPORTANT: run this cell first, then a message appears that you have to restart the session.
# in the notebook click on "restart session". Afterwards run all other cells.
# DO NOT RUN all cells immediately after opening the notebook! Run the first cell only. Then the rest.

# install packages
!apt-get -qq install -y poppler-utils tesseract-ocr libmagic1
!pip install "unstructured[all-docs]" python-magic llama-index==0.12.35 llama-index-vector-stores-chroma llama-index-embeddings-huggingface chromadb pdf2image pytesseract gradio

Selecting previously unselected package poppler-utils.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.12_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.12) ...
Setting up poppler-utils (22.02.0-2ubuntu0.12) ...
Processing triggers for man-db (2.10.2-1) ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 24.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of llama-index-vector-stores-chroma to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-index-embeddings-huggingface to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (p

In [1]:
# install package that lets you use Groq or OpenAI
!git clone https://github.com/dgaida/llm_client.git
%cd llm_client
!pip install .
%cd ..

Cloning into 'llm_client'...
remote: Enumerating objects: 497, done.
remote: Counting objects: 100% (189/189), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 497 (delta 109), reused 117 (delta 48), pack-reused 308 (from 1)
Receiving objects: 100% (497/497), 1.10 MiB | 21.27 MiB/s, done.
Resolving deltas: 100% (283/283), done.
/content/llm_client
Processing /content/llm_client
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 6.6 MB/s eta 0:00:00
  Created wheel for llm-client: filename=llm_client-0.2.0-py3-none-any.whl size=34021 sha256=a35573c5264f4f6ed5b912f7c7d08dd2546c14bcf837f0bfc6f8af6684148993
  Stored in directory: /tmp/pip-ephem-wheel-cache-o7jlm9tr/wheels/d0/e6/fc/4b9383b6c127c40ef39f8e988f6338269b2cc0a669deb9aea1
Successfully built llm-client
/content


In [2]:
# --- Import required libraries ---
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
import gradio as gr
from llm_client import LLMClient, LLMClientAdapter

import sys

sys.path.append("/content/llm_client/notebooks")

import utils

# --- Step 1: Set up PDF file path ---
# Put your PDF files in a folder named "pdfs" in the current directory
PDF_DIR = "pdfs"

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'validate_default' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'validate_default' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(


In [3]:
# --- Step 2: Load and parse PDFs with uMiner (via UnstructuredReader) ---
all_documents = utils.read_pdf_files_with_unstructured_reader(PDF_DIR)

print(f"Loaded {len(all_documents)} documents")
for doc in all_documents[:3]:
    print(doc.text[:300])  # Zeige ersten Ausschnitt

File:.ipynb_checkpoints
File:00_Formalitaeten_DBS.pdf


Loaded 1 documents
Datenbanksysteme (DBS für INF, WI, MI, ITM)

Vorlesung Wintersemester 2025/2026 - Formalitäten

10.12.25

Datenbanksysteme (AI WI MI ITM)

Prof. Dr. Johann Schaible | Prof. Dr. Birgit Bertelsmeier

Seite 1

Fachbereich 10 | Institut für Informatik | Gummersbach

Volle Anerkennung von Datenbanken 1 i


In [9]:
print(f"Loaded {len(all_documents)} documents \n")
for doc in all_documents[:3]:
    # print(doc.text[:300])  # Zeige ersten Ausschnitt
    print(type(doc))

Loaded 1 documents 

<class 'llama_index.core.schema.Document'>


In [ ]:
# split documents into smaller chunks for better retrieval
node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=100)  # 512, 50 --> creates an overlap between chunks and thereby increases the ability to find information in der Chunks

In [ ]:
# this is the embedding model
# models can be found here: https://huggingface.co/spaces/mteb/leaderboard
embed_model = HuggingFaceEmbedding(model_name="intfloat/e5-small-v2")
# embed_model = HuggingFaceEmbedding(model_name = "sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
# client that uses openAI or Grog API, here groq API
client = LLMClient(api_choice="groq")

In [ ]:
# Adapter erzeugen
adapter = LLMClientAdapter(client=client)

In [ ]:
# Apply settings globally - LlamaIndex uses this Settings object internally
Settings.llm = adapter  # None
Settings.embed_model = embed_model
Settings.node_parser = node_parser

In [ ]:
# --- Step 4: Create vector store and index ---
index = utils.create_chromadb_vector_store_and_index(all_documents)

In [ ]:
# --- Step 5: Create query engine ---
query_engine = index.as_query_engine()

In [ ]:
# this function does everyting.
# takes the query of the user and embeds it into the vector space,
# does a semantic search in the vector database (compares vectors)
# returns the text of the most similar vector
# passes this text as context to the LLM together with the query
# retrieves the response of the LLM and returns it as a string
# llama_index does this all internally using the Settings object.
def chat_with_pdf(query, history=None):
    return utils.safe_query_engine_call(query_engine, query)

In [ ]:
print(chat_with_pdf("Welche Module gibt es in dem Studiengang?"))

In [ ]:
# --- Step 7: Build Gradio Interface ---
# This creates a simple web UI for asking questions
chat_ui = gr.ChatInterface(
    fn=chat_with_pdf,
    title="PDF RAG Chatbot",
    description="Ask questions about the content of your PDF documents.",
    theme="default",
    examples=["What is this PDF about?", "Summarize the second section."],
)

chat_ui.launch()